In [ ]:
from IPython.display import display, Markdown, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
from pathlib import Path, PosixPath

# Visual inspection of the beam based alignment measurement

## Notebook setup

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import os.path
import matplotlib.pyplot as plt

from dataclasses import dataclass
# %matplotlib notebook
%matplotlib inline

## Loading data

In [ ]:
data_dir = Path(os.getcwd()) / ".." / "bact_analysis_bessyii"/ "scripts"
data_dir = Path(os.path.normpath(data_dir))
data_dir

In [ ]:
os.listdir(data_dir)

In [ ]:
data_dir

In [ ]:
ls -ltr '/home/waheed/workspace/bact/custom/bessyii/analysis/bact_analysis_bessyii/scripts'

In [ ]:
uid = 'd9ebee40-5e37-4584-bd60-42308121a81f'

In [ ]:
twiss_db = xr.load_dataset(
 os.path.join(data_dir, '..', 'bba', 'bessyii_twiss_thor_scsi_twin.nc')
)
twiss_db

In [ ]:
import epics

In [ ]:
epics.PV('Pierre:DT:beam:nu:y')

That's something perhaps to be handled by a (caching) database?

In [ ]:
estimated_angles = xr.load_dataset(os.path.join(data_dir, f'estimated_angles_{uid}.nc'))
offsets = xr.load_dataarray(os.path.join(data_dir, f'offsets_{uid}.nc'))
preprocessed_measurement_data = xr.load_dataset(os.path.join(data_dir, f'preprocessed_measurement_data_{uid}.nc'))

In [ ]:
preprocessed_measurement_data

In [ ]:
estimated_angles

In [ ]:
len(estimated_angles.attrs['bpm_names'])

In [ ]:
preprocessed_measurement_data.y_pos.shape

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[16, 8])
ax.plot(
    twiss_db.ds, estimated_angles.orbit.sel(name='q3m2t8r', plane='y')[:-1]
)

In [ ]:
bpm_names_lower = [name.lower() for name in preprocessed_measurement_data.coords['bpm'].values]
ds_bpm = twiss_db.ds.sel(pos=bpm_names_lower)
len(ds_bpm)

In [ ]:
fig, axes = plt.subplots(3, 1, sharex=True, figsize=[16, 12])
pscale = 1e6
ax, ax_diff, ax_twiss  = axes
sel = preprocessed_measurement_data.y_pos.sel(name="q3m2t8r")
ref = None
for y in sel:
    line, = ax.plot(ds_bpm, y * pscale, '.-', linewidth=.5)
    if ref is None:
        ref = y
        continue
    ax_diff.plot(ds_bpm, (y.values - ref.values) * pscale, '-', color=line.get_color())
ax_twiss.plot(
    twiss_db.ds, - estimated_angles.orbit.sel(name='q3m2t8r', plane='y')[:-1] * pscale, '--'
)
ax.set_ylabel("bpm")
ax_twiss.set_ylabel("orbit deviation from twiss")
ax_diff.set_ylabel("diff")
for ax in axes:
    ax.grid(True)
len(ref)

In [ ]:
for n1, n2 in zip(estimated_angles.attrs['bpm_names'], [name.lower() for name in preprocessed_measurement_data.coords["bpm"].values]):
    if n1 != n2:
        print(n1 == n2, n1, n2)

In [ ]:
from databroker import catalog
db = catalog["heavy_local"]
run = db[uid]
stream =run.primary
dataX = stream.read()


In [ ]:
dataX

In [ ]:
def extract(item):
    return  item["y"]["pos_raw"]

fig, ax = plt.subplots(1, 1, figsize=[16, 6])
for elems  in dataX.bpm_elem_data.values:
    ax.plot(
     dataX.bpm_ds[0],  np.array([extract(item) for item in  elems]), '.-', linewidth=.5
    )
    break


In [ ]:

fig, ax = plt.subplots(1, 1, figsize=[16, 8])
ax.plot(
    twiss_db.ds, estimated_angles.orbit.sel(name='q3m2t8r', plane='y')[1:-1]
)

In [ ]:
extract_data(dataX.bpm_elem_data[0, 0])

In [ ]:
json.loads(tmp[1:-1])

In [ ]:
fig, ax = plt.subplots()
ax.plot(


## Display overview of quadrupole offsets

Not developed here: e.g. display a table

| element name | dx | dy | mse | mae |
|:-------------|---:|---:|---:|---:|
| q1m1d1r      | 0.03 | 0.02 | 0.43 | 0.24 |

### Analyse the result for one quadrupole

Visual analysis:

* reproduce fit data (shall one reuse the code ?)

    * read estimated angle for one quadrupole (both planes)
    * read ideal kick
    * scale ideal kick with angle

* read measurement data for one quadrupole

   * subtract the offset
   * see if the deviations correspond to the measured ones

In [ ]:
element_name = "q1m1d1r"

Select the magnet data to analyse

In [ ]:
estimated_angles


In [ ]:
estimated_angles_for_magnet = estimated_angles.sel(name=element_name)
estimated_angles_for_magnet

## Prepare comparison between measured data and calculated data

The orbit stored in the data was calculated for a steerer placed at the element producing a
kick of some value $\vartheta$

The visual analysis of measured data to estimated fit requires

1. reduce the offsets of the BPM"s from the measurements
   These offsets are stored in the fit data results

2. scale the orbit which was made for some value $\vartheta$ with the fit value $\vartheta_f$ by

   $$
       orbit \cdot \vartheta_f / \vartheta
   $$

3. plot the data:

    1. all together to see if the offsets are sensible
    2. measured and calculated with the bpm offsets reduced



In [ ]:
ref_angle = estimated_angles_for_magnet.orbit.attrs['theta']

### Plot the orbit as stored just to see

In [ ]:
pscale = 1e6
fig, ax = plt.subplots(1, 1)
ax.plot(
    estimated_angles_for_magnet.ds, estimated_angles_for_magnet.orbit.sel(plane="x") * pscale
)
ax.set_ylabel("dx [mm]")
ax.set_xlabel("ds [m]")

Need to retrieve the bpm names: want to extract the offset from the fit
Luckily its an attribute

In [ ]:
bpm_names = estimated_angles.attrs['bpm_names']

In [ ]:
bpm_offsets = estimated_angles_for_magnet.fit_params.sel(parameter=bpm_names).rename(parameter="pos")
bpm_offsets

In [ ]:
bpm_offsets.sel(plane="x", result="value")

In [ ]:
orbit_at_bpm = estimated_angles_for_magnet.orbit.sel(pos=bpm_names)
ds_at_bpm = estimated_angles_for_magnet.ds.sel(pos=bpm_names)
ds_at_bpm

In [ ]:
estimated_angles_for_magnet

In [ ]:
#bpm_offsets.sel(plane="x", result="value").rename(fit_parameters="pos")

In [ ]:
pscale = 1e6
fig, axes = plt.subplots(2, 1, sharex=True, figsize=[16, 12])
ax, ax_diff = axes
# difference to orbit for quadrupole with piggy pack power converter at 0
# well not exactly ... for where it would be if the magnet was ideal and
# had no  hysteresis etc.

ax.plot(ds_at_bpm, bpm_offsets.sel(plane="x", result="value") * pscale, 'b.')
for cnt, x in enumerate(estimated_angles_for_magnet.x_pos):
    line, = ax.plot(ds_at_bpm, x * pscale, '.-')
    dx = x.values - bpm_offsets.sel(plane="x", result="value").values
    ax_diff.plot(ds_at_bpm, dx * pscale, '.-', color=line.get_color())

ax_diff.plot(ds_at_bpm, orbit_at_bpm.sel(plane="y") * pscale / 10, '.-', color='k')

ax.set_ylabel("x [um]")
ax_diff.set_ylabel("dx [um]")
ax_diff.set_xlabel("ds [m]")

In [ ]:
pscale = 1e6
fig, axes = plt.subplots(2, 1, sharex=True, figsize=[16, 12])
ax, ax_diff = axes
# difference to orbit for quadrupole with piggy pack power converter at 0
# well not exactly ... for where it would be if the magnet was ideal and
# had no  hysteresis etc.

ax.plot(ds_at_bpm,  bpm_offsets.sel(plane="y", result="value") * pscale, 'b.')
for cnt, y in enumerate(estimated_angles_for_magnet.y_pos):
    line, = ax.plot(ds_at_bpm, y * pscale, '.-')
    dy = y.values - bpm_offsets.sel(plane="y", result="value").values
    ax_diff.plot(ds_at_bpm, dy * pscale, '.-', color=line.get_color())

#ax_diff.plot(ds_at_bpm, orbit_at_bpm.sel(plane="y") * pscale / 10, '.-', color='k')

ax.set_ylabel("y [mm]")
ax_diff.set_ylabel("dy [mm]")
ax_diff.set_xlabel("ds [m]")

In [ ]:
plt.plot(ds_at_bpm, orbit_at_bpm.sel(plane="y") * 1e6, '.-', color='k')

In [ ]:
pscale = 1e6
fig, ax = plt.subplots(1, 1)

# difference to orbit for quadrupole with piggy pack power converter at 0
# well not exactly ... for where it would be if the magnet was ideal and
# had no  hysteresis etc.
dx = orbit_at_bpm.sel(plane="x")
ax.plot(
     ds_at_bpm, dx * pscale, 'b.-',
    ds_at_bpm,  bpm_offsets.sel(plane="x", result="value") * pscale, 'g.'
)
ax.set_ylabel("dx [mm]")
ax.set_xlabel("ds [m]")

In [ ]:
offsets.sel(result="error", plane="x")

## Cross check to data provided by digital twin

### Twiss data as provided by the twin

In [ ]:
import epics

In [ ]:
offsets.sel(name="q1m1d1r", plane="x") * 1e3

In [ ]:
orbit_x_pv =  epics.PV("Pierre:DT:beam:orbit:x")
orbit_y_pv =  epics.PV("Pierre:DT:beam:orbit:y")
beta_x_pv =  epics.PV("Pierre:DT:beam:beta:x")
beta_y_pv =  epics.PV("Pierre:DT:beam:beta:y")
mu_x_pv =  epics.PV("Pierre:DT:beam:nu:x")
mu_y_pv =  epics.PV("Pierre:DT:beam:nu:y")
s_pv =  epics.PV("Pierre:DT:beam:s")
orbit_names = epics.PV("Pierre:DT:beam:names")

In [ ]:
mu_x_pv.get();

In [ ]:
data = np.transpose([pv.get() for pv in (orbit_x_pv, orbit_y_pv, beta_x_pv, beta_y_pv, mu_x_pv, mu_y_pv, s_pv)])
twin_twiss_df = pd.DataFrame(data, columns=["x", "y", "beta_x", "beta_y", "mu_x", "mu_y", "ds"], index=orbit_names.get())
twin_twiss_df.tail()

### Comparing used orbit distortom

The one from the twin data to the one used during calculation

In [ ]:
element_name

In [ ]:
from bact_math_utils.distorted_orbit import closed_orbit_distortion
orbit_x = closed_orbit_distortion(
    twin_twiss_df.beta_x, twin_twiss_df.mu_x * 2*np.pi, tune=twin_twiss_df.mu_x.iloc[-1],
    beta_i=twin_twiss_df.loc[element_name, 'beta_x'],
    mu_i=twin_twiss_df.loc[element_name, 'mu_x'] * 2*np.pi,
    theta_i=1e-5,
)

In [ ]:
pscale = 1e6
fig, ax = plt.subplots()
ax.plot(twin_twiss_df.ds, orbit_x * pscale, '-b',
         estimated_angles_for_magnet.ds, estimated_angles_for_magnet.orbit.sel(plane="x") * pscale, '-g',
        ds_at_bpm,orbit_at_bpm.sel(plane='x') * pscale, '+r',
        linewidth=.2
        )
ax.set_xlabel('s [m]')
ax.set_ylabel('x [um]')

In [ ]:
bpm_names = [name for name in twin_twiss_df.index if name[:3] == "bpm"  ]
bpm_names, len(bpm_names)

In [ ]:
bpm_indices = [ name[:3] == "bpm"  for name in twin_twiss_df.index]

In [ ]:
twin_twiss_df.index[bpm_names]

### middle offset to beam offset

In [ ]:
factor = 5.4
fig, ax = plt.subplots(figsize=[16, 12])
ax.plot(
    twin_twiss_df.ds, orbit_x * pscale * factor, '.-b',
    twin_twiss_df.ds[bpm_indices], orbit_x[bpm_indices] * pscale * 5.3, 'r+-',
    ds_at_bpm, orbit_at_bpm.sel(plane="x") * pscale * factor, 'k+',
    #ds_at_bpm, bpm_offsets.sel(plane="x", result="value") * pscale, 'gx-',
    twin_twiss_df.ds, twin_twiss_df.x * pscale, '.r',
    linewidth=.2
)



In [ ]:
from bact_analysis.transverse.calc import for_fitting_reference_orbit
dir(bact_analysis.transverse.calc)

In [ ]:
dx_q1_pv.get() - df.loc['q1m1d1r', :] * 1e3


In [ ]:
(df*1000).plot()

In [ ]:
plt.plot(
    estimated_angles_for_magnet.ds.sel(pos=df.index), df.x * 1000
)

In [ ]:
dx_q1_pv = epics.PV("Pierre:DT:q1m1d1r:x:rdbk")

In [ ]:
set(df.index.values).difference(set(estimated_angles_for_magnet.ds.coords['pos'].values)),
set(estimated_angles_for_magnet.ds.coords['pos'].values).difference(set(df.index.values))

In [ ]:
df.index[:2]